In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [26]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [27]:
#Preprocess the data
# drop irrelevant columns

data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [28]:
data.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [29]:
## Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0


In [30]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo= OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])

In [31]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [32]:
geo_encoded_df= pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [33]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [34]:
#save the encoders
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [35]:
#divide the dataset into train and test
X=data.drop('Exited',axis=1)
y=data['Exited']

#split the data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#scale the features
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [36]:
X_test

array([[-5.12501721e-01,  9.09111664e-01, -6.77299309e-01, ...,
        -1.02020406e+00,  1.73668197e+00, -5.63491843e-01],
       [-2.36046598e-01,  9.09111664e-01,  3.84298354e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [-4.61306328e-01, -1.09997489e+00,  4.80807232e-01, ...,
        -1.02020406e+00, -5.75810666e-01,  1.77464858e+00],
       ...,
       [ 8.59534812e-01, -1.09997489e+00,  7.70333868e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [ 4.70449825e-01,  9.09111664e-01, -9.66825944e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [-1.84851205e-01,  9.09111664e-01, -1.73715981e-03, ...,
        -1.02020406e+00,  1.73668197e+00, -5.63491843e-01]])

In [37]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN IMPLEMENTATION

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [39]:
#Build our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #1st hidden layer connect with input layer
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid') #output layer
]
)

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
import tensorflow
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss= tensorflow.keras.losses.BinaryCrossentropy()
loss

In [42]:
## compile the model
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
#set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


log_dir='logs/fit' +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [44]:
#Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [45]:
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4462 - accuracy: 0.8099 - val_loss: 0.3976 - val_accuracy: 0.8270
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3871 - accuracy: 0.8416 - val_loss: 0.3614 - val_accuracy: 0.8445
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3559 - accuracy: 0.8519 - val_loss: 0.3468 - val_accuracy: 0.8520
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3438 - accuracy: 0.8570 - val_loss: 0.3452 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3389 - accuracy: 0.8586 - val_loss: 0.3428 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3369 - accuracy: 0.8600 - val_loss: 0.3404 - val_accuracy: 0.8630
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3327 - accuracy: 0.8606 - val_loss: 0.3398 - val_accuracy: 0.8605

In [46]:
model.save('model.h5')

c:\Users\Priyal Chugh\Desktop\Churn_modelling\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [47]:
#Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:
%tensorboard --logdir logs/fit20240827-221541

Reusing TensorBoard on port 6007 (pid 25512), started 1 day, 22:00:34 ago. (Use '!kill 25512' to kill it.)